In [ ]:
!pip install pyarabic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 2.4 MB/s eta 0:00:00


In [ ]:
!pip install clean-text # for removing emojis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 1.8 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171031 sha256=9ff34f8cfff67ac41bf68d26ed724913f33d6b5aabd5be0c7a4e03ac3bdd9518
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji


In [ ]:
!pip install emoji

In [ ]:
import re
import string
from string import punctuation

import pandas as pd
import numpy as np
import plotly.express as px

from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

import tensorflow as tf
import keras
from keras import Sequential
from keras.layers import Dense , Embedding, Input

import pyarabic

# Download the Dataset

In [ ]:
!kaggle datasets download -d abedkhooli/arabic-100k-reviews

Dataset URL: https://www.kaggle.com/datasets/abedkhooli/arabic-100k-reviews
License(s): copyright-authors
 82% 14.0M/17.0M [00:00<00:00, 74.7MB/s]
100% 17.0M/17.0M [00:00<00:00, 74.8MB/s]


In [ ]:
!unzip arabic-100k-reviews.zip

Archive:  arabic-100k-reviews.zip
  inflating: ar_reviews_100k.tsv     


# Data Exploration

In [ ]:
df = pd.read_csv('/content/ar_reviews_100k.tsv', sep='\t')
df.head()

,label,text
0,Positive,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,Positive,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,Positive,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,Positive,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,Positive,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   99999 non-null  object
 1   text    99999 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB


In [ ]:
df['label'].value_counts()

,count
label,
Positive,33333
Mixed,33333
Negative,33333


In [ ]:
# cleaning
def remove_url(list):
  cleaned = re.sub('https?://\S+|www\.\S+', '', list)
  cleaned = re.sub('\[.*?\]', '', cleaned)
  cleaned = re.sub('[%s]' % re.escape(punctuation), '', cleaned)
  cleaned = re.sub('<.*?>+', '', cleaned)
  cleaned = re.sub('\n', '', cleaned)
  cleaned = re.sub('\w*\d\w*', '', cleaned)
  return cleaned

df['text'] = df['text'].apply(lambda x: remove_url(x))

## Remove emojis

In [ ]:
import emoji

def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U0001F1E0-\U0001F1FF"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

df['text'] = df['text'].apply(lambda x: remove_emojis(x))


# Tokenazation

In [ ]:
vectorizer = CountVectorizer()
tok = vectorizer.build_tokenizer()

df['text'] = df['text'].apply(tok)

# Dropping missing values

In [ ]:
maxLength = df['text'].apply(len).max()
maxLength

1531

In [ ]:
meanLength = df['text'].apply(len).mean()
meanLength

51.84742847428474

In [ ]:
minLength = df['text'].apply(len).min()
minLength

0

In [ ]:
no_text = df[df['text'].apply(len) == minLength]
df.drop(no_text.index, inplace=True)

# Commen words

most commin words in all dataset

In [ ]:
top = Counter([word for lists in df['text'] for word in lists])
top_10 = top.most_common(10)
top_10 = pd.DataFrame(top_10)

fig = px.bar(top_10, x=0, y=1 , labels={'0':'words', '1':'count'})
fig.show()

common words in Positive class

In [ ]:
filtered_pos = df[df['label'] == 'Positive']
top = Counter([word for lists in filtered_pos['text'] for word in lists])
top_10 = top.most_common(10)
top_10 = pd.DataFrame(top_10)

fig = px.bar(top_10, x=0, y=1 , labels={'0':'words', '1':'count'})
fig.show()

Common words in negative class

In [ ]:
filtered_neg = df[df['label'] == 'Negative']
top = Counter([word for lists in filtered_neg['text'] for word in lists])
top_10 = top.most_common(10)
top_10 = pd.DataFrame(top_10)

fig = px.bar(top_10, x=0, y=1 , labels={'0':'words', '1':'count'})
fig.show()

In [ ]:
df['text']

,text
0,ممتاز نوعا ما . النظافة والموقع والتجهيز والشا...
1,أحد أسباب نجاح الإمارات أن كل شخص في هذه الدول...
2,هادفة .. وقوية. تنقلك من صخب شوارع القاهرة الى...
3,خلصنا .. مبدئيا اللي مستني ابهار زي الفيل الاز...
4,ياسات جلوريا جزء لا يتجزأ من دبي . فندق متكامل...
...,...
99994,معرفش ليه كنت عاوزة أكملها وهي مش عاجباني من ا...
99995,لا يستحق ان يكون في بوكنق لانه سيئ . لا شي. لا...
99996,كتاب ضعيف جدا ولم استمتع به. فى كل قصه سرد لحا...
99997,مملة جدا. محمد حسن علوان فنان بالكلمات، والوصف...


# Data Cleaning and Preprocessing

## Filter by text length

In [ ]:
# >= 50 and <= 500
df = df[df['text'].apply(lambda x: 50 <= len(x) <= 500)]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26852 entries, 1 to 99988
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   26852 non-null  object
 1   text    26852 non-null  object
dtypes: object(2)
memory usage: 629.3+ KB


## Delete mixed labels

In [ ]:
mixed = df[df['label'] == 'Mixed']

df.drop(mixed.index, inplace = True)

In [ ]:
df.shape

(17527, 2)

## Arabic stopwords

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords

stops = set(stopwords.words('arabic'))
print(stops)

def remove_stops(list):
  return [w for w in list if w not in stops]

df['text'] = df['text'].apply(lambda x: remove_stops(x))

{'فلان', 'ؤ', 'كانون', 'هَجْ', 'اربعون', 'هناك', 'كلاهما', 'كأنّ', 'عاشر', 'لسن', 'عاد', 'بخٍ', 'أربعة', 'أينما', 'تسعين', 'كن', 'الألى', 'دينار', 'اللتين', 'هل', 'قرش', 'ساء', 'ارتدّ', 'الذي', 'ثاء', 'خاء', 'منذ', 'يورو', 'إياهما', 'أجمع', 'حبيب', 'ومن', 'ثمَّ', 'أنى', 'ذَيْنِ', 'أربعاء', 'يفعلون', 'ة', 'ثلاثاء', 'نحن', 'سبع', 'هذه', 'واو', 'تسع', 'بيد', 'أو', 'همزة', 'مع', 'قام', 'حاء', 'ولو', 'أخٌ', 'عجبا', 'آها', 'تلكم', 'آهاً', 'كيفما', 'إيانا', 'والذي', 'لعلَّ', 'مئة', 'م', 'آي', 'مايو', 'هاك', 'لعمر', 'أي', 'ليسا', 'مكانكم', 'شباط', 'حجا', 'خامس', 'بك', 'إلّا', 'أيها', 'تسعون', 'قبل', 'ثمة', 'بَلْهَ', 'لكما', 'أبدا', 'ث', 'أربعمئة', 'هاتين', 'أقل', 'تخذ', 'ألا', 'هلّا', 'كرب', 'سبعمائة', 'سابع', 'نفس', 'فإن', 'أفريل', 'اللذين', 'بما', 'ته', 'ستين', 'حرى', 'كِخ', 'خلف', 'أيلول', 'شَتَّانَ', 'إمّا', 'ثمانون', 'له', 'يمين', 'سبتمبر', 'بطآن', 'مادام', 'أولاء', 'لستم', 'عشرة', 'ثلاث', 'درهم', 'فو', 'حيث', 'ظلّ', 'كأنما', 'أوه', 'تفعلون', 'استحال', 'سحقا', 'أمام', 'سنتيم', 'ستون', 'أب

In [ ]:
# print most common words after removing stop words
top = Counter([word for lists in df['text'] for word in lists])
top_10 = top.most_common(10)
top_10 = pd.DataFrame(top_10)

fig = px.bar(top_10, x=0, y=1 , labels={'0':'words', '1':'count'})
fig.show()

## Removing Arabic diacritics (Tashkeel) and elongated letters (Tatweel):

In [ ]:
from pyarabic.araby import strip_tashkeel, strip_tatweel

def remove_tashkeel(list):
  return [strip_tashkeel(w) for w in list]


def tatweel(list):
  return [strip_tatweel(w) for w in list]

df['text'] = df['text'].apply(lambda x: remove_tashkeel(x))
df['text'] = df['text'].apply(lambda x: tatweel(x))

In [ ]:
# test
mylist = ["الْعَرَبِيّةُ" , 'هيا']

remove_tashkeel(mylist)

['العربية', 'هيا']

## Stemming

In [ ]:
from nltk.stem.arlstem import ARLSTem

stemmer = ARLSTem()

def stemmer_apply(list):
  return [stemmer.stem(w) for w in list]

df['text'] = df['text'].apply(lambda x: stemmer_apply(x))


## Normalizing Hamza

In [ ]:
import pyarabic.araby as araby


def normalize_hamza(list):
  return [araby.normalize_hamza(w, method="tasheel") for w in list]

df['text'] = df['text'].apply(lambda x: normalize_hamza(x))


## Joining tokens after cleaning

In [ ]:
df['text'] = df['text'].apply(lambda x: ' '.join(x))

In [ ]:
df['text']

,text
1,سبب جاح امار شخص دول عشق راب نحب امار مض فكر ص...
3,خلص مبديي للي مستني بهر زي فيل ازرق ميقراش احس...
11,عاد دهشني منذر باسلوب كتاب سلسل حدث حبك موامر ...
12,كتاب حال فضايي جنون بعي كتاب نويري كتر كون سير...
17,عندم فرح نظن نهاي لفرح عندمانحز نظن نهاي لحز ك...
...,...
99966,ممممممم فرج فود رغم ستفادت كتب كبير قراءتي لاك...
99977,عدم وفر غرف علم انه وجد شي حظوري فندق خبرن موض...
99984,خايب امل بشد رفع لذي عرف هذه حكاي هزيل لتي يكن...
99987,بدو نني وحيد كوكب ارض مبتلي بقراء رواي عقيم رو...


# Tokenization, Padding, and Data Splitting

# Tokinazation

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tok = Tokenizer()
tok.fit_on_texts(df['text'])

seq = tok.texts_to_sequences(df['text'])


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = max(len(x) for x in seq)
padded_seq = pad_sequences(seq, maxlen=max_len, padding='post')


# Label encoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
labels = le.fit_transform(df['label'])

# Train - test split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_seq, labels, test_size=0.2, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(14021, 455)
(14021,)
(3506, 455)
(3506,)


# Building the RNN Model

In [ ]:
vocab_size = len(tok.word_index) + 1
embedding_size = 128
input_length = padded_seq.shape[1]


model = tf.keras.Sequential([
    # Input(shape=(input_length,)),
    tf.keras.layers.Embedding(vocab_size, 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

# Defining Batch Size, Creating Datasets, and Training the Model

In [ ]:
batch_size = 50


train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train) )
test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test) )

train_data = train_data.batch(batch_size)
test_data = test_data.batch(batch_size)

In [ ]:
history = model.fit(train_data, epochs=10, validation_data=test_data)

Epoch 1/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 529s 2s/step - accuracy: 0.5491 - loss: 0.6661 - val_accuracy: 0.8018 - val_loss: 0.4347
Epoch 2/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 460s 2s/step - accuracy: 0.8344 - loss: 0.3828 - val_accuracy: 0.8274 - val_loss: 0.4081
Epoch 3/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 461s 2s/step - accuracy: 0.9155 - loss: 0.2317 - val_accuracy: 0.8192 - val_loss: 0.4716
Epoch 4/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 491s 2s/step - accuracy: 0.9599 - loss: 0.1362 - val_accuracy: 0.8149 - val_loss: 0.5719
Epoch 5/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 477s 2s/step - accuracy: 0.9757 - loss: 0.0932 - val_accuracy: 0.8118 - val_loss: 0.6051
Epoch 6/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 455s 2s/step - accuracy: 0.9831 - loss: 0.0653 - val_accuracy: 0.8043 - val_loss: 0.6701
Epoch 7/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 455s 2s/step - accuracy: 0.9887 - loss: 0.0481 - val_accuracy: 0.8009 - val_loss: 0.7761
Epoch 8/10
281/281 ━━━━━━━━━━━━━━━━━━━━ 507s 2s/step - accuracy: 0.9942 - loss: 0.0265 - val_accu

# Model Evaluation

In [ ]:
# Plot training and validation accuracy
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', Validation'], loc='upper left')
plt.show()

# Model Inference

In [ ]:
new_text = ["السلام عليكم ورحمة الله "]

new_sequences = tok.texts_to_sequences(new_text)
new_padded_seq = pad_sequences(new_sequences, maxlen=input_length, padding='post')

predictions = model.predict(new_padded_seq)


for i, prediction in enumerate(predictions):
    print(f"Text: {new_text[i]}")
    print(f"Predicted Probability: {prediction[0]}")
    predicted_label = 1 if prediction[0] > 0.5 else 0
    print(f"Predicted Label: {predicted_label}")
